In [2]:
import csv
import time
import pandas as pd

import numpy as np
import os
from PIL import Image

np.random.seed(1234) # 난수 발생 패턴 고정
def randomize(): np.random.seed(time.time()) # 난수 발생 패턴 재설정

In [3]:
RND_MEAN = 0 # 정수분포 난숫값의 평균
RND_STD = 0.0030 # 정수분포 난숫값의 표준편차

LEARNING_RATE = 0.001 # 학습률

In [4]:
def load_steel_dataset():
    
    global input_cnt, output_cnt, data, every_data, data_20, data_8, show_data
    
    input_cnt = 401
    output_cnt = 5

    os.chdir('./20_data')
    file_names_20=os.listdir() # 20 x 20 학습데이터가 들어있는 폴더 안에 있는 파일을 모두 읽어옴
    os.chdir('../8_data')
    file_names_8=os.listdir() # 8 x 8 학습데이터가 들어있는 폴더 안에 있는 파일을 모두 읽어옴
    
    data_20=np.zeros([len(file_names_20),input_cnt+output_cnt])
    data_8=np.zeros([len(file_names_8),input_cnt+output_cnt])
    # data = np.zeros([len(file_names_20)+len(file_names_8), input_cnt+output_cnt]) #학습 데이터 n개 x 406([픽셀값 400] + [20,8 사이즈 판단 1] + [1~5정답값 5개]) 배열
    show_data=np.zeros([len(file_names_20)+len(file_names_20),20,20]) #학습 데이터 입력값 n x 400 배열 (그냥 이미지 픽셀)

    data_idx=0;

    # Every_Data 안에 있는 파일의 픽셀값과 정답을 data배열에 넘김
    for filename in file_names_20:
        
        # 파일을 이미지로 받아오고 흑백 이미지의 픽셀 배열로 변환
        image=Image.open('../20_data/'+filename)
        image=image.convert('L')
        pix=np.array(image)
        pix=pix[1:,:]
        # show_data에 2차원 배열로 이미지 저장
        show_data[data_idx]=pix
        
        # data 배열의 :400 까지는 1차원 픽셀값 저장 / 400: 부터는 정답값 저장 
        pix=np.concatenate(pix).tolist()
        data_20[data_idx,:input_cnt-1] = pix
        answer=os.path.splitext(filename)[0][0]
        data_20[data_idx,input_cnt-1]=0 # 400번째 인덱스에 20 x 20 이미지라는 것을 표시
        data_20[data_idx,input_cnt-1+int(answer)]=1 # 401~405까지의 인덱스에는 1~5까지 값 중에 무엇인지 표시
        data_idx+=1
        
    data_idx=0
    
    for filename in file_names_8:
        
        # 파일을 이미지로 받아오고 흑백 이미지의 픽셀 배열로 변환
        image=Image.open('../8_data/'+filename)
        image=image.convert('L')
        pix=np.array(image)
        pix=pix[1:,:]
        
        # show_data에 2차원 배열로 이미지 저장
        show_data[data_idx]=pix
        
        # data 배열의 :400 까지는 1차원 픽셀값 저장 / 400: 부터는 정답값 저장 
        pix=np.concatenate(pix).tolist()
        data_8[data_idx,:input_cnt-1] = pix
        answer=os.path.splitext(filename)[0][0]
        data_8[data_idx,input_cnt-1]=1 # 400번째 인덱스에 8 x 8 이미지라는 것을 표시
        data_8[data_idx,input_cnt-1+int(answer)]=1 # 401~405까지의 인덱스에는 1~5까지 값 중에 무엇인지 표시
        data_idx+=1
        
    every_data=np.concatenate((data_20,data_8),axis=0)

    data=_data
        
    os.chdir("../")

In [5]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt]) #가중치 행렬을 정규분포를 갖는 10 X 1 의 난숫값으로 초기화
    bias = np.zeros([output_cnt])

In [6]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0]) #data.shape[0]=데이터 갯수 1~데이터갯수만큼의 배열 생성
    np.random.shuffle(shuffle_map) # 무작위로 순서 섞기
    step_count = int(data.shape[0] * 0.8) // mb_size # 학습용 데이터와 테스트용 데이터 경계 나눔
    test_begin_idx = step_count * mb_size # 테스트 데이터 시작 인덱스
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]] #test_data 테스트 시작 인덱스부터의 data 배열
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:] #test_data 배열의 입력값과 그에 따른 목표값 각각 반환

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx]) # 한번의 학습 데이터셋이 학습될 때마다 학습 데이터를 바꿔줌
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]] # 10개의 데이터마다 한번씩 학습시킴
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:] #train_data 배열의 입력값과 그에 따른 목표값 각각 반환


In [7]:
def softmax(x):
    max_elem = np.max(x, axis=1)
    diff = (x.transpose() - max_elem).transpose()
    exp = np.exp(diff)
    sum_exp = np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()
    return probs

def softmax_derv(x, y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size, nom_size, nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n, i, j] = -y[n,i] * y[n,j]
            derv[n, i, i] += y[n,i]
    return derv

def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [8]:
def forward_postproc(output, y):
    entropy = softmax_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) 
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [9]:
def eval_accuracy(output, y):
    estimate = np.argmax(output, axis=1)
    answer = np.argmax(y, axis=1)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [10]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [11]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x) # 순전파 행렬 계산
    loss, aux_pp = forward_postproc(output, y) # 오차 계산 (평균제곱오차, 오차)
    accuracy = eval_accuracy(output, y) # 
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp) #제곱오차값을 통해 
    backprop_neuralnet(G_output, aux_nn) # 역전파 행렬 계산
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [12]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size) # 무작위로 데이터를 섞고 데이터 갯수를 학습 횟수로 나눈 값 반환
    test_x, test_y = get_test_data() #data의 나머지 20프로를 test_data의 입력값, 출력값으로 추출
     
    
    for epoch in range(epoch_count): # 학습 횟수만큼씩 돌아가면서
        losses, accs = [], [] # 각 epoch의 loss, accs 초기화
        
        for n in range(step_count): # 0~step_count까지
            train_x, train_y = get_train_data(mb_size, n) #0*10 ~ step_cout*10 까지 10개씩 학습시킴
            loss, acc = run_train(train_x, train_y) 
            losses.append(loss)
            accs.append(acc)
            
            
        if report > 0 and (epoch+1) % report == 0: # 하나의 데이터셋에 대해 학습이 끝나면
            acc = run_test(test_x, test_y) # get_test_data를 통해 얻는 test값들을 통해 테스트
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc)) # 횟수, 오차, 정확도 출력
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc)) # 마지막 학습에 대해 횟수, 오차, 정확도 출력

In [13]:
def steel_exec(_data,epoch_count=10, mb_size=10, report=1):
    load_steel_dataset(_data)
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [18]:
os.chdir('c:/Users/taebe/Desktop/Joe/Embeded/2022_2/인공지능/To Student 2022/Code/chap03')
steel_exec(data_20,10,10,1)

Epoch 1: loss=17.378, accuracy=0.242/0.167
Epoch 2: loss=17.712, accuracy=0.231/0.236
Epoch 3: loss=17.889, accuracy=0.223/0.250
Epoch 4: loss=18.198, accuracy=0.208/0.236
Epoch 5: loss=18.598, accuracy=0.192/0.167
Epoch 6: loss=17.978, accuracy=0.219/0.167
Epoch 7: loss=17.889, accuracy=0.223/0.250
Epoch 8: loss=17.624, accuracy=0.235/0.236
Epoch 9: loss=17.978, accuracy=0.219/0.278
Epoch 10: loss=18.421, accuracy=0.200/0.250

Final Test: final accuracy = 0.250
